# Laplace Approximation

In [ ]:
%pip install lightning-uq-box

## Theoretic Foundation

The Laplace Approximation was originally introduced by [MacKay, 1992](https://authors.library.caltech.edu/13793/1/MACnc92b.pdf). Then, the Laplace Approximation has been adapted to modern neural networks by [Ritter, 2018](https://discovery.ucl.ac.uk/id/eprint/10080902/1/kflaplace.pdf) and [Daxberger, 2021](http://proceedings.mlr.press/v139/daxberger21a/daxberger21a.pdf) and is an approximate Bayesian method. The goal of the Laplace Approximation is to use a second-order Taylor expansion around the fitted MAP estimate and yield a posterior approximation over the model parameters via a full-rank, diagonal or Kronecker-factorized approach. In order for the Laplace Approximation to be computationally feasible for larger network architectures, we use the [Laplace library](https://aleximmer.github.io/Laplace/) to include approaches, such as subnetwork selection that have been for example proposed by [Daxberger, 2021](http://proceedings.mlr.press/v139/daxberger21a/daxberger21a.pdf).

The general idea of the Laplace Approximation to obtain a distribution over the network parameters is to approximate the posterior with a Gaussian distribution centered at the MAP estimate of the parameters [Daxberger, 2021](http://proceedings.mlr.press/v139/daxberger21a/daxberger21a.pdf). In this setting, we define a prior distribution $p(\theta)$ over our network parameters. Because modern neural networks consists of millions of parameters, obtaining a posterior distribution over the weights $\theta$ is intractable. The LA takes MAP estimate of the parameters $\theta_{MAP}$ from a trained network $f_{\theta_{MAP}}(x) = \mu_{\theta_{MAP}}(x)$ and constructs a Gaussian distribution around it. The parameters $\theta_{MAP}$ are obtained by
$$
    \theta_{MAP} = \text{argmin} \mathcal{L}(\theta; D),
$$

where $\mathcal{L}$ is the mean squared error or also referred to as the $\ell^2$ loss, $\mathcal{L}(\theta; \mathcal{D}) := -\sum_{i=1}^n log(p(y_i|f_{\theta}(x_i)))$ and we chose the posterior $p(y_i|f_{\theta}(x_i))$ to be a Gaussian with constant variance $\sigma^2$, such that the loss is the mean squared error and a homoskedastic noise model is assumed. Then with Bayes Theorem, as in  [Daxberger, 2021](http://proceedings.mlr.press/v139/daxberger21a/daxberger21a.pdf), one can relate the posterior to the loss,

$$
    p(\theta|D) = p(D\vert\theta)p(\theta)/p(D)= \frac{1}{Z} exp(- \mathcal{L}(\theta; D)),
$$

with $Z = \int p(D\vert\theta)p(\theta) d\theta$. Now a second-order expansion of $\mathcal{L}$ around $\theta_{MAP}$ is used to construct a Gaussian approximation to the posterior $p(\theta|D)$:

$$
    -\mathcal{L}(\theta; D) \approx -\mathcal{L}(\theta_{MAP}; D)- \frac{1}{2}(\theta-\theta_{MAP}) (\nabla_{\theta}^2 \mathcal{L}(\theta; D)\vert \theta_{MAP}) (\theta-\theta_{MAP}).
$$

The term with the first order derivative is zero as the loss is evaluated at a minimum $\theta_{MAP}$ [Murphy, 2022](https://books.google.de/books?hl=de&lr=&id=OyYuEAAAQBAJ&oi=fnd&pg=PR27&ots=A7kHWDyx-9&sig=pCj6y7c54tW3dleHahn3kdu-JNw#v=onepage&q&f=false), and, further, one assumes that the first term is neglible as the loss is evaluated at $\theta = \theta_{MAP}$. Then taking the expontential of both sides allows to identify, after normalization, the Laplace approximation,

\begin{align*}
p(\theta|D) \approx \mathcal{N}(\theta_{MAP}, \Sigma) && \text{with} \qquad \Sigma =  (\nabla_{\theta}^2 \mathcal{L}(\theta; D)\vert \theta_{MAP})^{-1}.
\end{align*}


As the covariance is just the inverse Hessian of the loss, with $\theta_{MAP}\in \mathcal{R}^W$ and $H^{-1}\in \mathcal{R}^{W\times W}$, with $W$ being the number of weights, we get the posterior distribution

$$
    p(\theta|D)\approx \mathcal{N}(\theta_{MAP}, H^{-1}).
$$

The computation of the Hessian term is still expensive. Therefore, further approximations are introduced in practice, most commonly the Generalized Gauss-Newton matrix \cite{martens2020new}. This takes the following form:
$$
    H \approx \widetilde{H}=\sum_{n=1}^NJ_n^TH_nJ_n,
$$
where $J_n\in \mathcal{R}^{O\times W}$ is the Jacobian of the model outputs with respect to the parameters $\theta$ and $H_n\in\mathcal{R}^{O\times O}$ is the Hessian of the negative log-likelihood with respect to the model outputs, where $O$ denotes the model output size and $W$ the number of parameters. 

During prediction we cannot compute the full posterior predictive distribution but instead resort to approximations. One strategy is to do sampling $\theta_s \sim p(\theta|D)$ for $s \in \{1, ...,S\}$ to approximate the predictions, however, [Immer et al. 2021](https://arxiv.org/abs/2008.08400) suggested that a linearization of the form $f_{\theta}(x)=f_{\theta_{MAP}}(x)+ J_{\theta_{MAP}}(\theta-\theta_{MAP})$ works better in practice and this is also the default in the [Laplace library](https://aleximmer.github.io/Laplace/).

$$
    \hat{y}(x^{\star}) = \frac{1}{S} \sum_{s=1}^S f_{\theta_s}(x^{\star}),
$$

and obtain the predictive uncertainty by

$$
    \sigma^2(x^{\star}) = \sqrt{\frac{1}{S} \sum_{s=1}^S f_{\theta_s}(x^{\star})^2 - \hat{y}(x^{\star})^2+\sigma^2}.
$$

The implementation is a wrapper around a model from the fantastic [Laplace library](https://aleximmer.github.io/Laplace/) so all the available options for subnet strategies can be found in their docs.

## Imports

In [ ]:
import os
import tempfile
from functools import partial

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from laplace import Laplace
from lightning import Trainer
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import CSVLogger

from lightning_uq_box.datamodules import ToyHeteroscedasticDatamodule
from lightning_uq_box.models import MLP
from lightning_uq_box.uq_methods import DeterministicRegression, LaplaceRegression
from lightning_uq_box.viz_utils import (
    plot_calibration_uq_toolbox,
    plot_predictions_regression,
    plot_toy_regression_data,
    plot_training_metrics,
)

plt.rcParams["figure.figsize"] = [14, 5]

In [ ]:
seed_everything(0)  # seed everything for reproducibility

We define a temporary directory to look at some training metrics and results.

In [ ]:
my_temp_dir = tempfile.mkdtemp()

## Datamodule

To demonstrate the method, we will make use of a Toy Regression Example that is defined as a [Lightning Datamodule](https://lightning.ai/docs/pytorch/stable/data/datamodule.html). While this might seem like overkill for a small toy problem, we think it is more helpful how the individual pieces of the library fit together so you can train models on more complex tasks.

In [ ]:
dm = ToyHeteroscedasticDatamodule()

X_train, y_train, train_loader, X_test, y_test, test_loader = (
    dm.X_train,
    dm.y_train,
    dm.train_dataloader(),
    dm.X_test,
    dm.y_test,
    dm.test_dataloader(),
)

In [ ]:
fig = plot_toy_regression_data(X_train, y_train, X_test, y_test)

## Model

For our Toy Regression problem, we will use a simple Multi-layer Perceptron (MLP) that you can configure to your needs. For the documentation of the MLP see [here](https://readthedocs.io/en/stable/api/models.html#MLP).

In [ ]:
network = MLP(n_inputs=1, n_hidden=[50, 50], n_outputs=1, activation_fn=nn.Tanh())
network

For the Laplace model, we first train a plain deterministic model to obtain a MAP estimate of the weights via the standard MSE loss. Subsequently, we fit the Laplace Approximation to obtain an estimate of the epistemic uncertainty for predictions.

In [ ]:
deterministic_model = DeterministicRegression(
    model=network,
    optimizer=partial(torch.optim.Adam, lr=1e-2),
    loss_fn=torch.nn.MSELoss(),
)

## Trainer

Now that we have a LightningDataModule and base model, we can conduct training with a [Lightning Trainer](https://lightning.ai/docs/pytorch/stable/common/trainer.html). It has tons of options to make your life easier, so we encourage you to check the documentation.

In [ ]:
logger = CSVLogger(my_temp_dir)
trainer = Trainer(
    max_epochs=100,  # number of epochs we want to train
    logger=logger,  # log training metrics for later evaluation
    log_every_n_steps=1,
    enable_checkpointing=False,
    enable_progress_bar=False,
)

Training our model is now easy:

In [ ]:
trainer.fit(deterministic_model, dm)

## Training Metrics

To get some insights into how the training went, we can use the utility function to plot the training loss and RMSE metric.

In [ ]:
fig = plot_training_metrics(
    os.path.join(my_temp_dir, "lightning_logs"), ["train_loss", "trainRMSE"]
)

## Fit Laplace
We will utilize the great [Laplace Library](https://aleximmer.github.io/Laplace/) that allows you to define different flavors of Laplace approximations. For small networks like in this example, one can fit the Laplace approximation over all weights, but this is not feasible for large million-parameter networks. In those cases, on can resort to a "last-layer" approximation, where only the last layer weights are stochastic, while all other weights are deterministic. This behavior is controlled with the `subset_of_weights` parameter. This is chosen in combination with the structure of the Hessian that is fitted, see the `hessian_structure` parameter. Check their documentation for details. The Lightning-UQ-Box provides a wrapper so that the workflow is the same as with any other implemented UQ-Method.

One can also tune the prior precision and sigma noise values after the Laplace fitting procedure with the argument `tune_prior_precision_and_sigma=True` otherwise, the prediction will rely on the default `sigma_noise` values passed to the Laplace class for an estimate of aleatoric uncertainty under a homoscedastic noise assumption.

In [ ]:
la = Laplace(
    deterministic_model.model,
    "regression",
    subset_of_weights="last_layer",
    hessian_structure="full",
)


laplace_model = LaplaceRegression(laplace_model=la)

trainer = Trainer(default_root_dir=my_temp_dir)

## Prediction

For prediction we can either rely on the `trainer.test()` method or manually conduct a `predict_step()`. Using the trainer will save the predictions and some metrics to a CSV file, while the manual `predict_step()` with a single input tensor will generate a dictionary that holds the mean prediction as well as some other quantities of interest, for example the predicted standard deviation or quantile. The Laplace wrapper module will conduct the Laplace fitting procedure automatically before making the first prediction and will use it for any subsequent call. Originally, this was done through sampling and multiple forward passes, however, [Immer et al. 2021](https://arxiv.org/abs/2008.08400) showed that a linearization of the model achieves better performance in practice:
$f_{\theta}(x)=f_{\theta_{MAP}}(x)+ J_{\theta_{MAP}}(\theta-\theta_{MAP})$
and that is the current default implementation. However, arguments can be passed to the class or the individual predict step to choose between the following procedures of sampling as `pred_type="glm"` (linearization) or `pred_type="nn"` (sampling).

In [ ]:
trainer.test(laplace_model, dm)

In [ ]:
preds = laplace_model.predict_step(X_test)

## Evaluate Predictions

In [ ]:
fig = plot_predictions_regression(
    X_train,
    y_train,
    X_test,
    y_test,
    preds["pred"],
    preds["pred_uct"],
    epistemic=preds["epistemic_uct"],
    aleatoric=preds["aleatoric_uct"],
    title="Laplace Approximation",
)

For some additional metrics relevant to UQ, we can use the great [uncertainty-toolbox](https://uncertainty-toolbox.github.io/) that gives us some insight into the calibration of our prediction. For a discussion of why this is important, see ...

In [ ]:
fig = plot_calibration_uq_toolbox(
    preds["pred"].cpu().numpy(),
    preds["pred_uct"].numpy(),
    y_test.cpu().numpy(),
    X_test.cpu().numpy(),
)

## Additional Resources

[Daxberger et al. 2020](https://arxiv.org/abs/2010.14689) introduced a subnetwork selection strategy that turns selected weights "Bayesian" while keeping the rest of the network deterministic and show performance on par with deep ensembles.